In [51]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 50)

In [19]:
## online db
# https://cgmix.uscg.mil/IIR/IIRSearch.aspx

## data files
enforce_data = pd.read_excel('../data/misle/MISLE Enforcement Activities DT.xlsx')
fac_data = pd.read_excel('../data/misle/MISLE Facility Activities DT.xlsx')
vessel_data = pd.read_excel('../data/misle/MISLE Vessel Activities DT.xlsx')
incident_data = pd.read_excel('../data/misle/MISLE Incident Investigations DT.xlsx')
#scraped from incident_data.activity_id column values
scraped_briefs_data = pd.read_csv('../data/misle/scrape/misle-scraped-brief.csv')

Let's figure out which `incident_data.activity_id` values didn't fetch brief data. Then we can try to manually enter these to see if they in fact don't have briefs.

In [20]:
incident_activity_list = incident_data['Activity ID'].tolist()
scraped_activity_list = scraped_briefs_data.activity_id.tolist()

not_scraped = []
for activity_id in incident_activity_list:
    if activity_id not in scraped_activity_list:
        not_scraped.append(activity_id)

In [21]:
len(not_scraped)

1098

In the process of doing a random sample check, I found that several `activity_ids` were present in the online interface, with an attached brief that could have been scraped. So I'm going to have to try these ones that didn't resolve again.

Randomly selected activity_ids that did resolve when checked manually:
- 3702509
- 3691634
- 4548573

------

After rerunning the `activity_ids` that did not resolve on the first attempt, another 47 records were scraped. A subsequent random check of the second round `activity_ids` that did not resolve returned no records that resolved manually.

In [12]:
enforce_data.head()

,Case ID,Activity ID,MISLE Enforcement Activity ID,Activity Start Date,Originating Department,Owner Department,Controlling Department,Enforcement Activity Type,Party Class,Party Role,Penalty Amount Imposed,Penalty Amount Recommended,Probationary Period Months Imposed,Probationary Period Months Recommended,Probationary Suspension Months Imposed,Probationary Suspension Months Recommended,Outright Suspension Months Imposed,Outright Suspension Months Recommended
0,486041.0,3658917,3658917,2010-01-05,MSU HUNTINGTON (007767),MSU HUNTINGTON (007767),MSU HUNTINGTON (007767),Warning (Civil Penalty),Organization,UNSPECIFIED,0,0,0,0,0,0,0,0
1,471472.0,3662189,3662189,2010-01-11,MSU HUNTINGTON (007767),MSU HUNTINGTON (007767),CG HEARING OFFICE (000809),Administrative Civil Penalty (Class I),Organization,UNSPECIFIED,3000,5000,0,0,0,0,0,0
2,486168.0,3663897,3663897,2010-01-14,MSU PADUCAH (007766),MSU PADUCAH (007766),MSU PADUCAH (007766),Suspension and Revocation,Person,Subject of Investigation,0,0,0,0,0,0,0,0
3,486186.0,3667584,3667584,2010-01-21,MSU PADUCAH (007766),MSU PADUCAH (007766),MSU PADUCAH (007766),Notice of Violation (NOV),Organization,UNSPECIFIED,0,500,0,0,0,0,0,0
4,486748.0,3670688,3670688,2010-01-27,MSU HUNTINGTON (007767),MSU HUNTINGTON (007767),MSU HUNTINGTON (007767),Notice of Violation (NOV),Organization,UNSPECIFIED,500,0,0,0,0,0,0,0


In [13]:
fac_data.head()

,Activity ID,Activity Start Date,Originating Department,Activity Type,Facility Name,Primary ID,Primary ID Type,Facility Type,Facility Address,City,State (Address),Postal Code,Country,Facility Latitude,Facility Longitude,State (Physical Location)
0,3658288,2010-01-04,MSU PITTSBURGH (007764),Facility Inspection,Neville Chemical Company,PITMS422,Unique Id,Waterfront Facility,2800 Neville Road,Pittsburgh,PA,15225,US,40.505700,-80.102710,PA
1,3658584,2010-01-04,MSU PITTSBURGH (007764),Facility Inspection,ERGON West Virginia Inc.,PITMS408,Unique Id,Waterfront Facility,ERGON,Newell,WV,26050,US,40.602222,-80.633610,WV
2,3658459,2010-01-04,SECTOR OHIO VALLEY (007728),Facility Inspection,CHEVRON (LOUISVILLE),LOUMS010,Unique Id,Waterfront Facility,4401 Bells Lane,Louisville,KY,40211,US,38.233333,-85.834163,KY
3,3658420,2010-01-04,SECTOR OHIO VALLEY (007728),Facility Inspection,THE GLENMORE DISTILLERY (OWENSBORO),LOUMS049,Unique Id,Waterfront Facility,2100 Hwy 60 East,Owensboro,KY,42302-1069,US,37.777500,-87.087222,KY
4,3658272,2010-01-04,MSU PITTSBURGH (007764),Facility Inspection,Gordon Terminal Service Co,PITMS411,Unique Id,Waterfront Facility,GORDON TERMINAL SERV CO,MCKEES ROCKS,PA,15136,US,40.467778,-80.051115,PA


In [17]:
vessel_data.head()

,Activity ID,Activity Date,Department,Activity Type,Activity Status,Activity Close Date,Activity Duration (Days),MISLE Vessel ID,IMO Number,Official Number,Hull Number,Vessel Name,Vessel Class,Vessel Type,Vessel Subtype,Vessel Service,Flag Abbr.,Length,Default Gross Tons,Regulatory Gross Tons,ITC Gross Tons,Inspected,Documented,Latitude,Longitude
0,3662131,2010-01-04,SECTOR OHIO VALLEY (007728),Vessel Inspection,Closed - Approved Inspection,2010-01-11 14:09:47.000,7.0,338533,NaN,955274.0,89-2001,CHEM 442,Barge,Bulk Liquid Cargo (Tank) Barge,General 2,Tank Barge,US,290.0,1578.0,1578.0,NaN,X,X,35.181770,-90.061980
1,3662165,2010-01-04,SECTOR OHIO VALLEY (007728),Vessel Inspection,Closed - Approved Inspection,2010-01-11 14:37:26.000,7.0,467408,NaN,1061148.0,96-2881,CHEM 182,Barge,Bulk Liquid Cargo (Tank) Barge,General 2,Tank Barge,US,195.0,716.0,716.0,NaN,X,X,29.755133,-95.092000
2,3659470,2010-01-05,SECTOR OHIO VALLEY (007728),Uninspected Towing Vessel Exam,Closed - Approved Inspection,2010-04-13 13:50:53.999,98.0,953350,NaN,1212363.0,460-85,HARPER E,Towing Vessel,Pushing Ahead (Towboat),Fleeting,Towing Vessel,US,25.9,21.0,21.0,NaN,NaN,NaN,37.983333,-87.983333
3,3671453,2010-01-05,SECTOR OHIO VALLEY (007728),Boarding,Closed - Agency Action Complete,2010-01-28 11:11:18.000,23.0,228855,NaN,563110.0,3437,BARBARA,Towing Vessel,Pushing Ahead (Towboat),Linehaul,Towing Vessel,US,93.5,237.0,237.0,NaN,NaN,X,38.351667,-85.638333
4,3671472,2010-01-05,SECTOR OHIO VALLEY (007728),Boarding,Closed - Agency Action Complete,2010-01-28 11:26:31.000,23.0,143909,NaN,575824.0,NaN,D. & R. BONEY,Towing Vessel,Pushing Ahead (Towboat),Linehaul,Towing Vessel,US,128.0,483.0,483.0,NaN,NaN,X,38.345000,-85.626667


In [28]:
incident_data.head()

,Case ID,Activity ID,Activity Date,Originating Department,Owner Department,Controlling Department,Cargo Property Damage,Facility Property Damage,Other Property Damage,Vessel Property Damage,Total Property Damage,Total Persons at Risk,Total Persons Dead,Total Persons Missing,Total Persons Injured,Vessels Actual Total Loss,Vessels Constructive Total Loss,Vessels Damaged,Vessels Undamaged,Level of Investigation,IMO Designation,US Designation,Serious Marine Incident Designation,Latitude,Longitude,Activity Title,Case Title,Incident Involved
0,486351,3673761,2010-01-01,MSU PADUCAH (007766),COMMANDANT (CG-INV-3) (009107),COMMANDANT (CG-INV-3) (009107),0,0,200,1800,2000,0.0,0,0,0,0,0,2,1,Data Collection,Routine,Routine,Not a Serious Marine Incident,37.069980,-88.571300,EMMA KATE Allision,MS Allision/EMMA KATE/ Paducah 37 04.1988 N 08...,Allision
1,487363,3662467,2010-01-01,MSU HUNTINGTON (007767),COMMANDANT (CG-INV-3) (009107),COMMANDANT (CG-INV-3) (009107),0,0,0,0,0,1.0,0,0,1,0,0,0,2,Informal,Routine,Routine,Serious Marine Incident,38.192972,-81.370418,24HUN10/UTV BILL STILE/INJURY,MS Loss of Life/Injury/BILL STILE/ Pt Pleasant...,Personnel Casualty - Injury
2,486358,3669435,2010-01-01,MSU PADUCAH (007766),COMMANDANT (CG-INV-3) (009107),COMMANDANT (CG-INV-3) (009107),0,0,0,5000,5000,0.0,0,0,0,0,0,1,1,Data Collection,Routine,Routine,Not a Serious Marine Incident,37.078450,-88.571400,JACK WALKER / Allision,MS Allision/JACK WALKER/ Paducah 37 05.3796 N ...,Allision
3,487396,3662636,2010-01-02,MSU PITTSBURGH (007764),COMMANDANT (CG-INV-3) (009107),COMMANDANT (CG-INV-3) (009107),0,0,0,500,500,0.0,0,0,0,0,0,1,0,Informal,Routine,Routine,Not a Serious Marine Incident,39.809536,-79.918479,MRT 1902 SINKING CUMBERLAND MINE MONRVR MM 80.5,MS Capsize/ Monongahela River MM 80.5./021630Z...,Material Failure/Malfunction
4,486798,3659777,2010-01-06,MSU PADUCAH (007766),COMMANDANT (CG-INV-3) (009107),COMMANDANT (CG-INV-3) (009107),0,0,0,500,500,0.0,0,0,0,0,0,1,7,Data Collection,Routine,Routine,Not a Serious Marine Incident,37.682540,-89.513720,CAROL ANN PARSONAGE Break Away / Grounding,MS Breakaway/CAROL ANN PARSONAGE/ 37 40.9524 N...,Grounding


In [30]:
scraped_briefs_data.head()

,activity_id,incident_brief
0,6670206,Incident Brief Under Review
1,6636149,Incident Brief Under Review
2,6635800,Incident Brief Under Review
3,6634967,Incident Brief Under Review
4,6617461,Incident Brief Under Review


Let's join the `incident_data` and the `scraped_briefs_data`.

In [52]:
incidents_full = incident_data.merge(scraped_briefs_data, how='left', 
                                     left_on='Activity ID', right_on='activity_id')
incidents_full['activity_year'] = incidents_full['Activity Date'].map(lambda x: x.year)
serious_incidents = incidents_full.loc[incidents_full['Serious Marine Incident Designation'] == 'Serious Marine Incident']

In [54]:
pd.pivot_table(incidents_full, values='Activity ID', index='Serious Marine Incident Designation',
              columns=['activity_year'], aggfunc='count')

#('Serious Marine Incident Designation')['Activity ID'].count()

activity_year,2010,2011,2012,2013,2014,2015,2016,2017,2018
Serious Marine Incident Designation,,,,,,,,,
Not a Serious Marine Incident,405,402,379,359,398,336,249,229,334
Serious Marine Incident,36,40,23,31,35,29,27,36,45


In [60]:
serious2018 = serious_incidents.sort_values(['activity_year','Total Property Damage'], ascending=False).head(45)

In [61]:
pd.set_option('display.max_colwidth', -1)

for activity_id in serious2018['Activity ID'].tolist():
    display(incidents_full[incidents_full['Activity ID'] == activity_id].incident_brief)


3030    NaN
Name: incident_brief, dtype: object

3031    NaN
Name: incident_brief, dtype: object

3376    Incident Brief Under Review
Name: incident_brief, dtype: object

3151    On 13APR2018 at approximately 1500 hrs. the M/V CHARLEY WALLACE( CGB, 1000HP,ON 623276) was doing fleet work at the IL # 13 Fleet RDB MM 980 OHR when they were leaving the fleet downbound maneuvering back into the channel when the Port MDE had a reduction in propulsion and would not go over 700 RPM (Clutch speed). This caused the vessel to become pinned on its Starboard side against the fleet, and the vessel began taking on water and listing to Port. At that time the master came full ahead on the STRB MDE and was able to clear the fleet. The port list of the vessel caused by the intake of water continued, allowing a downflooding condition.  The vessel subsequently sunk approximately 100 ft from the IL # 13 Fleet RDB. All crew was able to exit the vessel in the water and was rescued by an assist vessel. Crewman did not experience any injuries from the incident, however a discharge of oil that created a sheen upon the navigable waters of the United States was observed as the vess

3155    At approximately 1105 on 18APR18 the M/V LEXINGTON (Tennessee Valley Towing, 5600 HP, 508204) was pushing 15 loaded barges (7 pig iron, 4 corn, 1 DRI, 1 bauxite, and 2 fly ash) U/B at MM 964 OHR when the head of the tow got caught in the current and pushed to port. The pilot on watch steered to stbd to try and catch the head but a face wire and wing wire broke on the STBD side. Crew attempted to catch more wires on tow but was unsuccessful and tow continued to drift to port as pilot was unsuccessful in stopping the head from swinging to port. Tow made contact with the long wall of Olmsted Lock causing 6 barges to break free of the tow. 4 barges were corraled without incident while 1 barge was pinned up against a construction barge at the Olmstead L&D and 1 barge pressed sideways straddling the #4 and #5 tainter gates at Olmstead L&D and sank.    \n\nCoast Guard investigation revealed the initiating event to be the material failure of the tow wires. Contributing factors include 

...

3335    Incident Brief Under Review
Name: incident_brief, dtype: object

3340    Incident Brief Under Review
Name: incident_brief, dtype: object

3359    Incident Brief Under Review
Name: incident_brief, dtype: object

3366    Incident Brief Under Review
Name: incident_brief, dtype: object

3373    Incident Brief Under Review
Name: incident_brief, dtype: object